In [1]:
!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=ac8cd6c85da9229093672ae137f05e020cc059f95ba5d64b28fbf7ececa91e87
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
# import numpy
from transformers import AutoTokenizer, AutoModel

In [4]:
# for summarization
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('UFNLP/gatortron-base')
model = AutoModel.from_pretrained('UFNLP/gatortron-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/379k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/713M [00:00<?, ?B/s]

In [6]:
file_path='/content/drive/MyDrive/Colab Notebooks/Testing Model (Tokanization issue)/Notes.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')
df.shape

(10000, 5)

In [7]:
text=' '.join(df[df.MRNO=='100000001456.00']['NOTE1'])

In [8]:
text_words=len(text.split(' '))
text_words

295

In [9]:
print(text)

CODE         SURGERY PROCEDURE                SITE
----         -----------------                ----
S0577        EUA & BIOPSY                      

SURGEON      Dr. Syed Adnan A. Shah
ANESTHETIST  Dr. Rehan  Amir

ANESTHESIA   Local Anesthesia


PRE OPERATIVE DIAGNOSIS
   osteomyelitis left maxilla

INCISION
   left upper alveolar margin

FINDINGS
   necrotic bone over left maxilla with sequestrum

PROCEDURES
   small flap raised from left maxilla retracting
   upper lip. squestrectomy done . for hemostasis
   surgicell placed

DRAINS
   N

OPERATIVE COMPLICATIONS
   none

Specimen Sent to Lab
NATURE OF         SITE OF SPECIMEN  SURGERY PROCEDURE
SPECIMEN
----------------  ----------------  ----------------
soet tissue       left maxilla      squestrectomy 
                                    and biopsy
bone              left maxilla      excision biopsy

PATIENT DESTINATION
   HOME

RECOMMENDATION
   discharge patient home after an hr.
   Advised cold liquids only for 24hr
     Med

In [10]:
# input_ids = tokenizer.encode(notes, return_tensors='pt')
#         embeddings = model(input_ids)[0] # correct way
#         # embeddings = input_ids #false positive

#         # Perform summarization
#         summary_sentences = summarize(notes, scores=list(embeddings.numpy()), ratio=0.5)
#         summarized_text = ' '.join([sentence[0] for sentence in summary_sentences])


IndentationError: unexpected indent (<ipython-input-10-39a2b67b8e25>, line 2)

In [11]:
input_ids = tokenizer.encode(text, return_tensors='pt')

In [12]:
len(input_ids[0])

228

In [13]:
embeddings = model(input_ids)[0]

In [14]:
embeddings

tensor([[[-0.0298, -0.0148, -0.0642,  ...,  0.3370, -0.7679, -0.0232],
         [-0.1024,  0.0148,  0.1601,  ..., -0.0037, -0.1333,  0.2164],
         [-0.2551,  0.2395, -0.0957,  ..., -0.2137, -0.4127,  0.1283],
         ...,
         [ 0.2442, -0.3068, -0.9221,  ...,  0.4913, -0.3461, -0.1888],
         [ 1.1907, -0.0568, -0.6756,  ..., -0.6122, -0.0119, -0.5151],
         [ 0.0257, -0.0634, -0.0335,  ...,  0.0074, -0.8489,  0.2056]]],
       grad_fn=<NativeLayerNormBackward0>)

In [15]:
list(embeddings.detach().numpy())

[array([[-0.0297896 , -0.01481909, -0.06418575, ...,  0.33702737,
         -0.7678774 , -0.02318048],
        [-0.10238569,  0.01481184,  0.16012989, ..., -0.00365302,
         -0.13325012,  0.21643494],
        [-0.25505388,  0.2394737 , -0.09573436, ..., -0.21365835,
         -0.4127019 ,  0.12825777],
        ...,
        [ 0.24423973, -0.30678168, -0.922124  , ...,  0.49126583,
         -0.3461388 , -0.18882959],
        [ 1.1906594 , -0.05677921, -0.6755606 , ..., -0.6122018 ,
         -0.01194997, -0.51505286],
        [ 0.02567287, -0.06344958, -0.03354734, ...,  0.00742696,
         -0.84886765,  0.2056236 ]], dtype=float32)]

In [16]:
from summa.summarizer import summarize
summary_sentences = summarize(text, scores=list(embeddings.detach().numpy()), ratio=0.5)

In [17]:
summarized_text = ' '.join([sentence[0] for sentence in summary_sentences])

In [18]:
summarized_text

'CODE         SURGERY PROCEDURE                SITE SURGEON      Dr. Syed Adnan A. ANESTHETIST  Dr. Rehan  Amir PRE OPERATIVE DIAGNOSIS osteomyelitis left maxilla left upper alveolar margin necrotic bone over left maxilla with sequestrum small flap raised from left maxilla retracting OPERATIVE COMPLICATIONS NATURE OF         SITE OF SPECIMEN  SURGERY PROCEDURE SPECIMEN soet tissue       left maxilla      squestrectomy  bone              left maxilla      excision biopsy HOME discharge patient home after an hr. Tab Augmentin 625mg, q8hr for 5 days Tab dicloran 5omg q12hr  for 5 days'

In [20]:
from summa.summarizer import summarize
print(summarize(text, ratio=0.5))

CODE         SURGERY PROCEDURE                SITE
SURGEON      Dr. Syed Adnan A.
ANESTHETIST  Dr. Rehan  Amir
PRE OPERATIVE DIAGNOSIS
osteomyelitis left maxilla
left upper alveolar margin
necrotic bone over left maxilla with sequestrum
small flap raised from left maxilla retracting
OPERATIVE COMPLICATIONS
NATURE OF         SITE OF SPECIMEN  SURGERY PROCEDURE
SPECIMEN
soet tissue       left maxilla      squestrectomy 
bone              left maxilla      excision biopsy
HOME
discharge patient home after an hr.
Tab Augmentin 625mg, q8hr for 5 days
Tab dicloran 5omg q12hr  for 5 days


**Long string testing**

In [21]:
text_2=' '.join(df[df.MRNO=='100000046580.00']['NOTE1'])
text_2_words=len(text_2.split(' '))
text_2_words

986

In [23]:
output_2=(summarize(text_2, ratio=0.5))

In [22]:
print(summarize(text_2, ratio=0.5))

physiotherapy
ex.session given physiotherapy
ex.session done physiotherapy
ex.session given pt.attented ex.session given
ex.session given ex.session given ex.session given ex.session given ex.session given pt.seen
ex.session given
mobilized in bed ex.session given 62 yr/male
mysthenia gravis and thymoma , sp thymectomy
on ventilator for a long time
regular ex.session given including: 
physiotherapy
pt.seen
exercise session given
mobilized in bed
physiotherapy
session given physiotherapy
session done
physiotherapy
exercise session done physiotherapy
session done
pt.mobilized also.
physiotherapy
pt.seen
exercise session given physiotherapy
pt.seen
regular exercise session given
has just moved out of i.c.u so not willing to be mobilized at this time,will be mob.in following days.
physiotherapy
pt.seen
physiotherapy
exercise session given physiotherapy
pt.seen
regular exercise session given
due to increased secretions pt.not willing to be out of bed.
physiotherapy
exercise session given ph

In [25]:
print(len(output_2.split(' ')))

484


In [26]:
484*2

968